In [1]:
import torch
import torch_geometric
from torch_geometric.datasets import KarateClub
from torch_geometric.nn import Node2Vec
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

# 加載示例數據集，這裡使用 Karate Club 數據集作為範例
dataset = KarateClub()
data = dataset[0]

# 設定 Node2Vec 模型的超參數
walk_length = 20
context_size = 10
embedding_dim = 128
walks_per_node = 10
p = 1.0
q = 1.0

# 初始化 Node2Vec 模型
model = Node2Vec(data.edge_index, embedding_dim, walk_length, context_size, walks_per_node, p, q)

# 將模型切換到訓練模式
model.train()

# 定義損失函數和優化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)

# 訓練模型
for epoch in range(4):
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to("cpu"), neg_rw.to("cpu"))
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# 將模型切換到評估模式
model.eval()

# 進行預測，獲得節點嵌入
with torch.no_grad():
    embeddings = model()


Epoch 0, Loss: 10.34440803527832
Epoch 1, Loss: 9.939845085144043
Epoch 2, Loss: 9.619058609008789
Epoch 3, Loss: 9.353951454162598


In [2]:
embeddings

Parameter containing:
tensor([[ 1.3044,  1.4876,  1.3902,  ...,  1.1390,  1.3349, -0.9400],
        [ 0.6600, -0.0955,  0.5711,  ...,  0.0175,  0.3856,  0.3038],
        [-1.1797,  0.4426,  1.3323,  ...,  0.9232,  2.0027, -0.6844],
        ...,
        [-1.0084, -0.0923,  0.4502,  ..., -0.1608, -0.8982, -1.4034],
        [-0.7189, -0.0336,  0.3899,  ..., -0.1815, -1.2432,  0.2965],
        [ 0.2004,  1.0188,  0.4942,  ..., -1.0302,  0.9087,  0.4428]],
       requires_grad=True)